In [1]:
from bs4 import BeautifulSoup
import requests
import time
from tqdm import tqdm

In [2]:
# Base URL
base_url = 'https://www.basketball-reference.com'

In [3]:
# Getting Team data
# Appending the '/teams/' to Base URL
url = base_url + '/teams/'

# Sending get request to the page
response = requests.get(url)

# Creating BeautifulSoup object
soup = BeautifulSoup(response.content, 'lxml')

# Getting all active team name from the BeautifulSoup object
team_name = soup.find(id='all_teams_active')

# Empty list for storing team hyperlink
team_url = []

# Getting all team's hyperlink
team_name = team_name.tbody.find_all(class_='full_table')

# Append the team's hyperlink to team_url[]
for i in team_name:
    team_url.append(i.a['href'])

# Writing team's hyperlink into a file
with open('.\\Data\\team_hyperlink.txt', 'w', encoding='utf8') as file_obj:
    for i in team_url:
        file_obj.write(i+'\n')

In [24]:
# Set to store player's name and href
player_data = set()

pbar = tqdm(total=len(team_url), desc='Total')
for i in team_url:
    for j in range(2010, 2022):
        # Get request to each team page
        data = requests.get(base_url+i+str(j)+'.html')
        # Create Beautiful Soup Object
        soup = BeautifulSoup(data.content, 'lxml')
        # Getting roster table from the website
        roster = soup.find("div", {"id": "div_roster"}).tbody
        
        # Extracting player href and names
        a_tags = roster.find_all('a')
        
        # Adding player name and href as a tuple to the 'player_data' set()
        for k in a_tags:
            if 'college' not in k['href']:
                player_data.add((k.text, k['href']))
        
        # Due to rate limit
        time.sleep(2)
        
    pbar.update(1)

Total:   7%|▋         | 2/30 [01:02<14:54, 31.96s/it]

AttributeError: 'NoneType' object has no attribute 'tbody'

In [23]:
# Writing the href and name in players.txt
with open('.\\Data\\players.txt', 'w', encoding='utf8') as file_obj:
    for i in player_data:
        name, url = i
        file_obj.write(name+'__'+url+'\n')